In [1]:
import pandas as pd

Merge all the data

In [2]:
# Read all the data
building1 = pd.read_csv("../data/schemas/warm_up/Building_1.csv", sep=",")
building2 = pd.read_csv("../data/schemas/warm_up/Building_2.csv", sep=",")
building3 = pd.read_csv("../data/schemas/warm_up/Building_3.csv", sep=",")
weather = pd.read_csv("../data/schemas/warm_up/weather.csv", sep=",")
carbon_intensity = pd.read_csv("../data/schemas/warm_up/carbon_intensity.csv", sep=",")

# Distinguish between the buildings
building1 = building1.add_suffix("_1")
building2 = building2.add_suffix("_2")
building3 = building3.add_suffix("_3")

# Merge
data = [building1, building2, building3, weather, carbon_intensity]
merged = pd.concat(data, axis=1)
merged.head()

,Month_1,Hour_1,Day Type_1,Daylight Savings Status_1,Indoor Temperature (C)_1,Average Unmet Cooling Setpoint Difference (C)_1,Indoor Relative Humidity (%)_1,Equipment Electric Power (kWh)_1,DHW Heating (kWh)_1,Cooling Load (kWh)_1,...,6h Outdoor Relative Humidity (%),12h Outdoor Relative Humidity (%),24h Outdoor Relative Humidity (%),6h Diffuse Solar Radiation (W/m2),12h Diffuse Solar Radiation (W/m2),24h Diffuse Solar Radiation (W/m2),6h Direct Solar Radiation (W/m2),12h Direct Solar Radiation (W/m2),24h Direct Solar Radiation (W/m2),kg_CO2/kWh
0,6,1,5,0,23.098652,-0.123570,61.086187,0.356839,0.055682,1.119216,...,72.980273,41.822361,83.230997,54.625927,116.842886,0.0,143.324335,1020.756093,0.0,0.402488
1,6,2,5,0,22.234742,0.012520,64.361378,0.345078,0.159338,1.469638,...,87.317655,37.910023,95.353948,78.375479,89.220504,0.0,300.361061,825.097292,0.0,0.382625
2,6,3,5,0,22.223061,0.000838,64.493415,0.338769,0.057004,1.458372,...,60.341453,30.190199,66.266409,259.822134,82.855932,0.0,196.691127,987.909330,0.0,0.369458
3,6,4,5,0,22.222251,0.000029,64.769702,0.334856,0.000000,1.337342,...,50.063925,31.519739,94.456370,248.631525,105.130119,0.0,465.483196,854.331825,0.0,0.367017
4,6,5,5,0,22.222236,0.000014,65.152120,0.348607,0.000000,1.163453,...,40.942579,33.119543,100.000000,272.825159,75.627986,0.0,569.203332,941.243325,0.0,0.374040


Drop the columns with constant values

In [3]:
merged_dropped = merged.drop(columns=["Month_1", "Month_2", "Month_3",
                                      "Daylight Savings Status_1", "Daylight Savings Status_2", "Daylight Savings Status_3",
                                      "Heating Load (kWh)_1", "Heating Load (kWh)_2", "Heating Load (kWh)_3",
                                      "HVAC Mode (Off/Cooling/Heating)_1", "HVAC Mode (Off/Cooling/Heating)_2", "HVAC Mode (Off/Cooling/Heating)_3"])

# Insert an index column
index = range(1, len(merged_dropped)+1)
merged_dropped.insert(loc=0, column="Index", value=index)

In [4]:
# Add a column with average solar generation for later prediction
merged_dropped["Avg solar generation"] = merged_dropped[["Solar Generation (W/kW)_1", "Solar Generation (W/kW)_2", "Solar Generation (W/kW)_3"]].mean(axis=1)

Split into train-test and normalize

In [5]:
total_length = merged_dropped.shape[0]
# last 3 days
SIZE_OF_TEST = 72
SIZE_OF_VALIDATION = 72

train = merged_dropped.head(total_length - SIZE_OF_TEST - SIZE_OF_VALIDATION)
validation = merged_dropped.iloc[total_length - SIZE_OF_TEST - SIZE_OF_VALIDATION: total_length - SIZE_OF_TEST] 
test = merged_dropped.tail(SIZE_OF_TEST)

# Ignore index when normalizing
columns = list(merged_dropped)
columns.remove("Index")

# Normalize
for column in columns:
    min_of_column = train[column].min()
    max_of_column = train[column].max()
    train[column] = (train[column] - min_of_column) / (max_of_column - min_of_column)
    validation[column] = (validation[column] - min_of_column) / (max_of_column - min_of_column)
    test[column] = (test[column] - min_of_column) / (max_of_column - min_of_column)

(720, 52)
(576, 52)
(72, 52)
(72, 52)


C:\Users\Matej\AppData\Local\Temp\ipykernel_11748\1884357144.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[column] = (train[column] - min_of_column) / (max_of_column - min_of_column)
C:\Users\Matej\AppData\Local\Temp\ipykernel_11748\1884357144.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation[column] = (validation[column] - min_of_column) / (max_of_column - min_of_column)
C:\Users\Matej\AppData\Local\Temp\ipykernel_11748\1884357144.py:25: SettingWithCopyWarning: 
A value is trying

Export the train and test sets

In [6]:
train.to_csv("../data/schemas/warm_up/TrainSet.csv", index=False)
validation.to_csv("../data/schemas/warm_up/ValidationSet.csv", index=False)
test.to_csv("../data/schemas/warm_up/TestSet.csv", index=False)

In [8]:
validation.head()

,Index,Hour_1,Day Type_1,Indoor Temperature (C)_1,Average Unmet Cooling Setpoint Difference (C)_1,Indoor Relative Humidity (%)_1,Equipment Electric Power (kWh)_1,DHW Heating (kWh)_1,Cooling Load (kWh)_1,Solar Generation (W/kW)_1,...,12h Outdoor Relative Humidity (%),24h Outdoor Relative Humidity (%),6h Diffuse Solar Radiation (W/m2),12h Diffuse Solar Radiation (W/m2),24h Diffuse Solar Radiation (W/m2),6h Direct Solar Radiation (W/m2),12h Direct Solar Radiation (W/m2),24h Direct Solar Radiation (W/m2),kg_CO2/kWh,Avg solar generation
576,577,0.000000,0.0,0.467447,0.568847,0.762258,0.003086,0.0,0.294832,0.0,...,0.488046,0.680018,0.085886,0.664934,0.0,0.166725,0.349403,0.0,0.419380,0.0
577,578,0.043478,0.0,0.340822,0.525564,0.907133,0.002510,0.0,0.409229,0.0,...,0.409663,0.530259,0.149267,0.555699,0.0,0.328624,0.367773,0.0,0.285699,0.0
578,579,0.086957,0.0,0.459823,0.382048,0.761972,0.002327,0.0,0.243847,0.0,...,0.163544,0.673081,0.228567,0.831410,0.0,0.371958,0.246345,0.0,0.197014,0.0
579,580,0.130435,0.0,0.438908,0.408417,0.803025,0.002137,0.0,0.210387,0.0,...,0.150047,1.000000,0.426786,0.612416,0.0,0.366639,0.324231,0.0,0.247500,0.0
580,581,0.173913,0.0,0.318857,0.539116,0.951972,0.001930,0.0,0.291284,0.0,...,0.434416,0.752468,0.624635,0.632975,0.0,0.415195,0.195349,0.0,0.329822,0.0
